In [733]:
import pathlib

import holoviews as hv
import hvplot.pandas # noqa
import numpy as np
import pandas as pd
import panel as pn
import datetime as dt

pn.extension(sizing_mode='stretch_width')

In [734]:
data = {'name':['A', 'B', 'C', 'D'],
        'open_time': [dt.datetime(2021, 7, 1, 0, 0, 0), dt.datetime(2021, 7, 2, 0, 0), dt.datetime(2021, 7, 3, 0, 0),dt.datetime(2021, 7, 3, 0, 0) ],
        'count':[27, 24, 22, 32],
        'size':[34, 56,67,78]
       }

df = pd.DataFrame(data)
df
df['open_time'] = pd.to_datetime(df['open_time']).dt.date
df


,name,open_time,count,size
0,A,2021-07-01,27,34
1,B,2021-07-02,24,56
2,C,2021-07-03,22,67
3,D,2021-07-03,32,78


In [735]:

df.set_index('open_time', inplace=True)
df

,name,count,size
open_time,,,
2021-07-01,A,27,34
2021-07-02,B,24,56
2021-07-03,C,22,67
2021-07-03,D,32,78


In [736]:
idf = df.interactive()

In [737]:
date_range = pn.widgets.DateRangeSlider(name='Date Range',
     start=df.index[0], end=df.index[-1])


In [738]:
filtered = (idf[
   (idf.index >= date_range.param.value_start) &
    (idf.index <= date_range.param.value_end)]
)

filtered.head()

#com_pipeline = (
#    idf[
#        (idf.open_time >= date_range.param.value_start) & (idf.open_time <= date_range.param.value_end)
#       ]
#       .groupby(['name', 'open_time'])[date_range].mean().to_frame()
#       .reset_index()
#       .sort_values(by='open_time')
#       .reset_index(drop=True)
#      )


In [739]:
def environment():
    try:
        get_ipython()
        return "notebook"
    except:
        return "server"
environment()

'notebook'

In [740]:
PALETTE = ["#ff6f69", "#ffcc5c", "#88d8b0","#58d8b0" ]
pn.Row(
    pn.layout.HSpacer(height=50, background=PALETTE[0]),
    pn.layout.HSpacer(height=50, background=PALETTE[1]),
    pn.layout.HSpacer(height=50, background=PALETTE[2]),
    pn.layout.HSpacer(height=50, background=PALETTE[3]),
)

Row(sizing_mode='stretch_width')
    [0] HSpacer(background='#ff6f69', height=50)
    [1] HSpacer(background='#ffcc5c', height=50)
    [2] HSpacer(background='#88d8b0', height=50)
    [3] HSpacer(background='#58d8b0', height=50)

In [741]:
if environment()=="server":
   theme="fast"
else:
   theme="simple"

In [742]:
itable = filtered.pipe(pn.widgets.Tabulator, pagination='remote', page_size=10, theme=theme)
itable

In [743]:
depth_hist = filtered.hvplot(y='count', kind='hist', responsive=True, min_height=200)

In [744]:
yaxis = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['count', 'size'],
    button_type='success'
)

In [745]:
ihvplot = filtered.hvplot(x='open_time', y=yaxis, by='name', color=PALETTE, line_width=1, height=400)
ihvplot



In [746]:
template = pn.template.FastListTemplate(
    title='Interactive DataFrame Dashboards', 
    sidebar=[date_range],
    main=[depth_hist.panel(), itable.panel(), ihvplot.panel()],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)
# template.show()
template.servable();
